In [2]:
import pandas as pd
import numpy as np

In [3]:
triple = pd.read_csv("Medication One Row Extraction/Three Combination One Row.csv")
single = pd.read_csv("Medication Extraction/Triple Combination.csv")

In [4]:
line1 = triple[triple["Line"] == 1]
triples = line1["Medication"].unique()
triples

array(['Bortezomib, Cyclophosphamide, Liposomal doxorubicin',
       'Bortezomib, Lenalidomide, Liposomal doxorubicin',
       'Bortezomib, Cyclophosphamide, Lenalidomide',
       'Adriamycin, Cyclophosphamide, Vincristine',
       'Melphalan, Prednisone, Thalidomide',
       'Cyclophosphamide, Prednisone, Thalidomide',
       'Adriamycin, Prednisone, Vincristine',
       'Bortezomib, Melphalan, Prednisone',
       'Cisplatin, Cyclophosphamide, Etoposide',
       'Bortezomib, Cyclophosphamide, Doxorubicin',
       'Bendamustine, Bortezomib, Cyclophosphamide'], dtype=object)

In [5]:
def getSingle(med):
    tripleT = triple[(triple["Line"] != 1) & (triple["Medication"] == med) & (triple["ID"] == ID)]
    if tripleT.shape[0] != 0:
        return tripleT.iloc[0]["PFS"]
    else:
        stop = single[(single["Line"] == 1) & (single["MedTx"] == med) & (single["ID"] == ID)]["DaysFromDxStop"].values[0]
        start = single[(single["Line"] == 1) & (single["MedTx"] == med) & (single["ID"] == ID)]["DaysFromDxStart"].values[0]
        return stop - start
        
def getDouble(med, med1, med2):
    tripleT = triple[(triple["Line"] != 1) & (triple["Medication"] == med) & (triple["ID"] == ID)]
    if tripleT.shape[0] != 0:
        return tripleT.iloc[0]["PFS"]
    else:
        medT = single[(single["Line"] == 1) & ((single["MedTx"] == med1) | (single["MedTx"] == med2)) & (single["ID"] == ID)]
        stop = medT["DaysFromDxStop"].max()
        start = medT["DaysFromDxStart"].min()
        return stop - start

In [6]:
for ABC in triples:
    abc = ABC.split(", ")
    A = abc[0]
    B = abc[1]
    C = abc[2]
    AB = abc[0] + ", " + abc[1]
    BC = abc[1] + ", " + abc[2]
    AC = abc[0] + ", " + abc[2]
    clm = ["ID", "PFS_" + ABC, "PFS_" + A, "PFS_" + B, "PFS_" + C, "PFS_" + AB, "PFS_" + BC, "PFS_" + AC]
    ABCpd = pd.DataFrame(columns = clm)
    medABC = line1[line1["Medication"] == ABC]
    medABC = medABC[["ID", "PFS"]]
    IDs = medABC["ID"]
    count = 0
    for ID in IDs:
        #A
        PFS_A = getSingle(A)
        #B
        PFS_B = getSingle(B)
        #C
        PFS_C = getSingle(C)
        #AB
        PFS_AB = getDouble(AB, A, B)
        #BC
        PFS_BC = getDouble(BC, B, C)
        #AC
        PFS_AC = getDouble(AC, A, C)
        PFS_ABC = medABC[medABC["ID"] == ID]["PFS"].values[0]
        s = pd.DataFrame(columns = clm)
        ABCpd.loc[count] = [ID, PFS_ABC, PFS_A, PFS_B, PFS_C, PFS_AB, PFS_BC, PFS_AC]
        count = count + 1
    ABCpd.to_csv("tri-regression/" + ABC + ".csv")